In [14]:
%load_ext autoreload
%autoreload 2

In [7]:
# create a pipeline from the etl_pipeline package
from pipeline_executor import PipelineExecutor
from nlp_analysis.NER import extract_entities
topic = 'Quantum Computing' # "quantumcomputing"AND"research"

# create a pipeline executor
pipeline_executor = PipelineExecutor()
quantum_df = pipeline_executor.execute(query=topic, max_articles=20, overwrite=True)

# print the first 5 rows of the dataframe
quantum_df.head()

Getting news article info: 100%|██████████| 50/50 [00:21<00:00,  2.38it/s]


,paragraph_index,engine,link,se_description,se_source,n3k_author,n3k_published,paragraph,title,body
0,0,Yahoo,https://finance.yahoo.com/news/10-quantum-comp...,"In this article, we will take a look at the 10...",Insider Monkey via Yahoo Finance,[Aima Zaheer],None,"In this article, we will take a look at the 10...",10 Quantum Computing Stocks Billionaires Are L...,"In this article, we will take a look at the 10..."
1,0,Yahoo,https://finance.yahoo.com/news/10-quantum-comp...,"In this article, we will take a look at the 10...",Insider Monkey via Yahoo Finance,[Aima Zaheer],None,"As reported in one of our articles earlier, th...",10 Quantum Computing Stocks Billionaires Are L...,"In this article, we will take a look at the 10..."
2,0,Yahoo,https://finance.yahoo.com/news/10-quantum-comp...,"In this article, we will take a look at the 10...",Insider Monkey via Yahoo Finance,[Aima Zaheer],None,Quantum computing has attracted the attention ...,10 Quantum Computing Stocks Billionaires Are L...,"In this article, we will take a look at the 10..."
3,0,Yahoo,https://finance.yahoo.com/news/10-quantum-comp...,"In this article, we will take a look at the 10...",Insider Monkey via Yahoo Finance,[Aima Zaheer],None,Transformative Impact of Quantum Computing,10 Quantum Computing Stocks Billionaires Are L...,"In this article, we will take a look at the 10..."
4,0,Yahoo,https://finance.yahoo.com/news/10-quantum-comp...,"In this article, we will take a look at the 10...",Insider Monkey via Yahoo Finance,[Aima Zaheer],None,The impact of quantum computing extends to the...,10 Quantum Computing Stocks Billionaires Are L...,"In this article, we will take a look at the 10..."


In [8]:
import pandas as pd

df = pd.read_csv('data/clean/QuantumComputing_20.csv')

In [9]:
df.head()

,paragraph_index,engine,link,se_description,se_source,n3k_author,n3k_published,paragraph,title,body
0,0,Yahoo,https://finance.yahoo.com/news/10-quantum-comp...,"In this article, we will take a look at the 10...",Insider Monkey via Yahoo Finance,['Aima Zaheer'],NaN,"In this article, we will take a look at the 10...",10 Quantum Computing Stocks Billionaires Are L...,"In this article, we will take a look at the 10..."
1,0,Yahoo,https://finance.yahoo.com/news/10-quantum-comp...,"In this article, we will take a look at the 10...",Insider Monkey via Yahoo Finance,['Aima Zaheer'],NaN,"As reported in one of our articles earlier, th...",10 Quantum Computing Stocks Billionaires Are L...,"In this article, we will take a look at the 10..."
2,0,Yahoo,https://finance.yahoo.com/news/10-quantum-comp...,"In this article, we will take a look at the 10...",Insider Monkey via Yahoo Finance,['Aima Zaheer'],NaN,Quantum computing has attracted the attention ...,10 Quantum Computing Stocks Billionaires Are L...,"In this article, we will take a look at the 10..."
3,0,Yahoo,https://finance.yahoo.com/news/10-quantum-comp...,"In this article, we will take a look at the 10...",Insider Monkey via Yahoo Finance,['Aima Zaheer'],NaN,Transformative Impact of Quantum Computing,10 Quantum Computing Stocks Billionaires Are L...,"In this article, we will take a look at the 10..."
4,0,Yahoo,https://finance.yahoo.com/news/10-quantum-comp...,"In this article, we will take a look at the 10...",Insider Monkey via Yahoo Finance,['Aima Zaheer'],NaN,The impact of quantum computing extends to the...,10 Quantum Computing Stocks Billionaires Are L...,"In this article, we will take a look at the 10..."


In [41]:
from nlp_analysis.word_wizard import WordWizard

# create a word wizard
word_wizard = WordWizard(df)
word_wizard.create_word_embeddings(columns=['body'])

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Creating embeddings for column(s): ['body']: 100%|██████████| 1/1 [00:08<00:00,  8.52s/it]


In [42]:
word_wizard.cluster_embeddings(column='body', method='silhouette')
word_wizard.df.head()

Optimal K: 14


,paragraph_index,engine,link,se_description,se_source,n3k_author,n3k_published,paragraph,title,body,sentences,body_word_embeddings,body_word_embeddings_cluster_13,body_word_embeddings_cluster_13_is_medoid,body_word_embeddings_cluster_14,body_word_embeddings_cluster_14_is_medoid,body_word_embeddings_cluster_15,body_word_embeddings_cluster_15_is_medoid
0,0,Yahoo,https://finance.yahoo.com/news/10-quantum-comp...,"In this article, we will take a look at the 10...",Insider Monkey via Yahoo Finance,['Aima Zaheer'],NaN,"In this article, we will take a look at the 10...",10 Quantum Computing Stocks Billionaires Are L...,"In this article, we will take a look at the 10...","[In this article, we will take a look at the 1...","[0.16420437395572662, -0.16303199529647827, -0...",10,True,10,True,2,True
1,0,Yahoo,https://finance.yahoo.com/news/10-quantum-comp...,"In this article, we will take a look at the 10...",Insider Monkey via Yahoo Finance,['Aima Zaheer'],NaN,"As reported in one of our articles earlier, th...",10 Quantum Computing Stocks Billionaires Are L...,"In this article, we will take a look at the 10...","[As reported in one of our articles earlier, t...","[0.16420437395572662, -0.16303199529647827, -0...",10,True,10,True,2,True
2,0,Yahoo,https://finance.yahoo.com/news/10-quantum-comp...,"In this article, we will take a look at the 10...",Insider Monkey via Yahoo Finance,['Aima Zaheer'],NaN,Quantum computing has attracted the attention ...,10 Quantum Computing Stocks Billionaires Are L...,"In this article, we will take a look at the 10...",[Quantum computing has attracted the attention...,"[0.16420437395572662, -0.16303199529647827, -0...",10,True,10,True,2,True
3,0,Yahoo,https://finance.yahoo.com/news/10-quantum-comp...,"In this article, we will take a look at the 10...",Insider Monkey via Yahoo Finance,['Aima Zaheer'],NaN,Transformative Impact of Quantum Computing,10 Quantum Computing Stocks Billionaires Are L...,"In this article, we will take a look at the 10...",[Transformative Impact of Quantum Computing],"[0.16420437395572662, -0.16303199529647827, -0...",10,True,10,True,2,True
4,0,Yahoo,https://finance.yahoo.com/news/10-quantum-comp...,"In this article, we will take a look at the 10...",Insider Monkey via Yahoo Finance,['Aima Zaheer'],NaN,The impact of quantum computing extends to the...,10 Quantum Computing Stocks Billionaires Are L...,"In this article, we will take a look at the 10...",[The impact of quantum computing extends to th...,"[0.16420437395572662, -0.16303199529647827, -0...",10,True,10,True,2,True


In [43]:
word_wizard.df['body_word_embeddings_cluster_13_is_medoid'].value_counts()

False    1022
True       13
Name: body_word_embeddings_cluster_13_is_medoid, dtype: int64

# The Rest

In [37]:
# for each entity run the pipeline (adding "quantum" to the query) and store the results in a dictionary
entity_results = {}
for entity in entities:
    entity_results[entity] = pipeline_executor.execute(query=f"'{entity}' AND '{topic}'", max_articles=100)


In [19]:
# check the results for the first entity
entity_results['Microsoft'].head()

,engine,link,se_description,se_source,n3k_author,n3k_published,title,body
0,Yahoo,https://seekingalpha.com/article/4603046-micro...,By leveraging the properties of quantum mechan...,Seeking Alpha,[Aseity Research],2023-05-11 11:03:43-04:00,Microsoft Stock: Leading The Quantum Computing...,Bartlomiej Wroblewski\n\nQuantum computing is ...
1,Yahoo,https://www.businessinsider.com/quantinuum-new...,Quantum computers are wondrous because they op...,Business Insider,[Julie Bort],2023-05-13 00:00:00,"I saw a quantum computer used by JPMorgan, wit...",Quantum computers are wondrous because they op...
2,Yahoo,https://www.marketwatch.com/press-release/quan...,Quantum Computing in Manufacturing Market is p...,Market Watch,[],None,Quantum Computing in Manufacturing Market Is T...,The MarketWatch News Department was not involv...
3,Yahoo,https://www.techtarget.com/searchstorage/tip/A...,"In 2017, Microsoft surprised attendees of its ...",SearchSecurity.com,[Published],None,A primer on quantum computing storage and memo...,"In 2017, Microsoft surprised attendees of its ..."
4,Yahoo,https://www.benzinga.com/pressreleases/23/05/n...,Quantum Computing Inc. initiates commercializa...,Benzinga,[],None,Quantum Computing Inc Announces First Quarter ...,Quantum Computing Inc. initiates commercializa...


In [21]:
# TODO summarize the results
import numpy as np
import matplotlib.pyplot as plt
import torch
import pandas as pd

In [22]:
torch.__version__

'2.0.1'

In [23]:
# Setup device agnostic code (Chooses NVIDIA or Metal backend if available, otherwise defaults to CPU)
if torch.cuda.is_available():
    device = torch.device("cuda")
    
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    
else:
    device = torch.device("cpu")

device

device(type='mps')

In [24]:
microsoft_df = entity_results['Microsoft']
apple_df = entity_results['Apple']
ibm_df = entity_results['IBM']

In [25]:
microsoft_df.shape, apple_df.shape, ibm_df.shape

((200, 8), (187, 8), (178, 8))

In [27]:
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm

import nltk

nltk.download('punkt')

/Users/quiquemz/.pyenv/versions/3.11.1/envs/capstone/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /Users/quiquemz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [28]:
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerN

In [29]:
def summarize_bart(
    column: pd.Series, tokenizer: BartTokenizer, model: BartForConditionalGeneration, min_len=50, max_len=100) -> str:
    
    long_text = column.str.cat()

    sentences = nltk.tokenize.sent_tokenize(long_text)

    # initialize
    length = 0
    chunk = ""
    chunks = []
    count = -1
    for sentence in sentences:
        count += 1
        combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

        if combined_length  <= 1024: # if it doesn't exceed  -tokenizer.max_len_single_sentence-
            chunk += sentence + " " # add the sentence to the chunk
            length = combined_length # update the length counter

            # if it is the last sentence
            if count == len(sentences) - 1:
                chunks.append(chunk) # save the chunk
            
        else: 
            chunks.append(chunk) # save the chunk
            # reset 
            length = 0 
            chunk = ""

            # take care of the overflow sentence
            chunk += sentence + " "
            length = len(tokenizer.tokenize(sentence))

    # inputs
    inputs = [tokenizer(chunk, return_tensors="pt").to(device) for chunk in chunks]

    # print summary
    outputs = []
    for input in tqdm(inputs):
        output = model.generate(**input, num_beams=2, min_length=min_len, max_length=max_len)
        outputs.append(tokenizer.decode(*output, skip_special_tokens=True))

    return " ".join(outputs)


In [30]:
# Generate summaries for Microsoft and Apple based on News Titles
microsoft_titles_summary = summarize_bart(microsoft_df["title"], tokenizer, model)
apple_titles_summary = summarize_bart(apple_df["title"], tokenizer, model)
ibm_titles_summary = summarize_bart(ibm_df["title"], tokenizer, model)

Token indices sequence length is longer than the specified maximum sequence length for this model (1485 > 1024). Running this sequence through the model will result in indexing errors
  0%|          | 0/3 [00:00<?, ?it/s]/Users/quiquemz/.pyenv/versions/3.11.1/envs/capstone/lib/python3.11/site-packages/transformers/generation/utils.py:719: UserWarning: MPS: no support for int64 repeats mask, casting it to int32 (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/Repeat.mm:236.)
  input_ids = input_ids.repeat_interleave(expand_size, dim=0)
/Users/quiquemz/.pyenv/versions/3.11.1/envs/capstone/lib/python3.11/site-packages/transformers/generation/utils.py:719: UserWarning: torch.cumsum supported by MPS on MacOS 13+, please upgrade (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/UnaryOps.mm:264.)
  input_ids = input_ids.repeat_interleave(expand_size, dim=0)
/Users/quiquemz/.pyenv/versi

In [31]:
microsoft_titles_summary

'Microsoft just made a huge, far-from-certain bet on nuclear fusion. I saw a quantum computer used by JPMorgan, with Microsoft and Nvidia as partners, and it was...Quantum Computing in Manufacturing Market Is Thriving Worldwide During Forecast Period 2029. 7 Legacy Tech Stocks Primed for a Comeback. Only Humility Can Save Us From AI. IBM and more team up to boost quantum tech community in Ireland. Microsoft takes a Quantum Leap Towards Sustainable Energy with Helion’s Nuclear Fusion Power Purchase Deal. Microsoft and Johnson Matthey join forces to speed up hydrogen fuel cell innovation with Azure Quantum. Microsoft and Classiq collaborate to offer researchers and educators accelerated quantum algorithm design. IBM, Microsoft, and Google Race to Close Quantum Skills Gap.'

In [32]:
apple_titles_summary

"IonQ Announces Pat Tang as New VP of Research & Development. ChatGPT developer OpenAI is the No. 1 chat app in the world, according to a new report. The chat app was founded in 2009 and is the first of its kind. Apple's quantum computing chips could be used in cars, planes, and more. Apple to use quantum computing to replace magnetic wire in its Macbook Pro and iPad Pro lines. Apple will use quantum chips to power its next-generation iWatch and iPad mini. Apple's new iWatch will be the first of its kind to run on a quantum processor. Apple is set to unveil a new iPhone 5S, 5C, 5S Plus and 5C. Is This the New Age of Computing? Apple's M2 Chip Gives New MacBook Air a Speed BoostDump Passwords on Your iPhone. Apple's New iOS 16 Feature Is More Secure.Eight leading quantum computing companies in 2020.IBM Tops 2018 Patent List as A.I. and Quantum Computing Gain Prominence. Apple's A15 Bionic chip powers iPhone 13 with 15 billion transistors. IBM and Japan join hands in the development of q

In [33]:
ibm_titles_summary

'IBM unveils new watsonx, AI and data platform. President Biden to visit Westchester to talk debt limit and impacts. Scientists create particles that remember their pasts. IBM and Microsoft are leading the quantum computing arms race. The company is also working with Google, Facebook and others.'

In [36]:
# Generate summaries for Microsoft and Apple based on News article bodies
microsoft_bodies_summary = summarize_bart(microsoft_df["body"], tokenizer, model)
apple_bodies_summary = summarize_bart(apple_df["body"], tokenizer, model)
ibm_bodies_summary = summarize_bart(ibm_df["body"], tokenizer, model)

100%|██████████| 207/207 [09:43<00:00,  2.82s/it]


In [58]:
# Summarize the long summary
def sub1024_summ(text: str, tokenizer: BartTokenizer, model: BartForConditionalGeneration, min_len=50, max_len=100) -> str:
    inputs = tokenizer([text], max_length=1024, return_tensors="pt").to(device)
    summary_ids = model.generate(inputs["input_ids"], num_beams=2, min_length=min_len, max_length=max_len)
    return tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]


In [65]:
sub1024_summ(microsoft_bodies_summary, tokenizer, model, min_len=100, max_len=300)

'Quantum computing is a disruptive technology that has the potential to revolutionize industries ranging from finance to healthcare. Microsoft stands out as a strong contender due to their innovative approach to quantum computing. IBM and Google are exploring superconducting qubits, while other companies like IonQ and Honeywell are experimenting with trapped ions. Microsoft is also pursuing topological qubits in its quantum computing work, a different way of going about quantum systems. QCI is engaged in a number of beta-tests to demonstrate the effectiveness of its computing capabilities.'

In [66]:
sub1024_summ(apple_bodies_summary, tokenizer, model, min_len=100, max_len=300)

"IonQ Announces Pat Tang as New VP of Research & Development (VP R&D) Tang joins IonQ with over 23 years of technology experience, most recently as an Engineering Vice President at Amazon Lab126. Tang has a diverse background that includes multi-touch sensor architecture, RF design and reliability engineering. Apple is opening a new development base in Israel to develop new Apple silicon chips for future Macs. Apple's new M2 Pro and M2 Max processors are 20% faster than their predecessors."

In [67]:
sub1024_summ(ibm_bodies_summary, tokenizer, model, min_len=100, max_len=300)

'Quantum computing is a disruptive technology that has the potential to revolutionize industries ranging from finance to healthcare. Google, IBM and other tech giants are developing quantum computers using superconductor qubits. Microsoft is also pursuing topological qubits in its quantum computing work. IBM has established an ecosystem of over 450,000 users with access to more than 20 quantum computers via the cloud. The more qubits that are quantum-mechanically linked together via entanglement, the more computations it can perform.'

In [ ]:
# ngl... not the best summaries I think